In [1]:
# Colab では requests は入っていますが、指示通り pip も実行しておきます
!pip -q install requests

import requests
from getpass import getpass
from pprint import pprint


In [4]:
print("Google Maps APIキーを入力します")
api_key = getpass("Google Maps APIキー")

print("続いて、駅名を入力します")
station_name = input("駅名: ").strip()

if not station_name:
    raise ValueError("駅名が空です。もう一度実行して駅名を入力してください。")


Google Maps APIキーを入力します
Google Maps APIキー··········
続いて、駅名を入力します
駅名: 名古屋駅


In [5]:
GEOCODE_URL = "https://maps.googleapis.com/maps/api/geocode/json"

geo_params = {
    "address": station_name,  # そのまま駅名でOK（例：新宿駅、梅田駅）
    "key": api_key,
    "language": "ja"
}

geo_res = requests.get(GEOCODE_URL, params=geo_params)
geo_json = geo_res.json()

# 途中確認（必要ならコメントを外して中身を見てください）
# pprint(geo_json)

status = geo_json.get("status")
if status != "OK" or not geo_json.get("results"):
    raise RuntimeError(f"ジオコーディングに失敗しました: status={status}, results={geo_json.get('results')}")

location = geo_json["results"][0]["geometry"]["location"]
lat, lng = location["lat"], location["lng"]

print(f"取得した座標: {station_name} → lat={lat}, lng={lng}")


取得した座標: 名古屋駅 → lat=35.170915, lng=136.8815369


In [6]:
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

places_params = {
    "location": f"{lat},{lng}",
    "radius": 500,           # 500m
    "type": "restaurant",    # レストラン
    "language": "ja",
    "key": api_key,
}

places_res = requests.get(PLACES_URL, params=places_params)
places_json = places_res.json()

# 途中確認（必要ならコメントを外して中身を見てください）
# pprint(places_json)

status = places_json.get("status")
if status != "OK":
    raise RuntimeError(f"Places API エラー: status={status}, error_message={places_json.get('error_message')}")

results = places_json.get("results", [])
print(f"\n入力した駅から半径500メートル以内のレストラン情報を出力（{len(results)}件）")

if not results:
    print("該当するレストランが見つかりませんでした。キーワードや場所を変えて再実行してください。")
else:
    for i, r in enumerate(results, start=1):
        name = r.get("name", "（名称不明）")
        rating = r.get("rating", "—")
        # Nearby Search は 'vicinity' に簡易住所が入ることが多い
        address = r.get("vicinity") or r.get("formatted_address") or "住所不明"
        print(f"{i:02d}. {name}\n    評価: {rating}\n    住所: {address}")



入力した駅から半径500メートル以内のレストラン情報を出力（20件）
01. 飛騨牛一頭家 馬喰一代 名古屋WEST
    評価: 4.1
    住所: 名古屋市西区牛島町６−２４ アクロスキューブ名古屋 2F
02. 中国料理 梨杏
    評価: 4.3
    住所: 名古屋市中村区名駅１丁目１−４ 名古屋マリオットアソシアホテル 18階
03. 和食・酒 えん 名古屋JRセントラルタワーズ店
    評価: 4
    住所: 名古屋市中村区名駅１丁目１−４ １３Ｆ
04. 南翔饅頭店 ミッドランドスクエア店
    評価: 3.8
    住所: 名古屋市中村区名駅４丁目７−１ ミッドランドスクエア 4F
05. すし 桂
    評価: 3.6
    住所: 名古屋市中村区名駅１丁目１−４ 13F
06. 日本料理 名古屋浅田
    評価: 4.4
    住所: 名古屋市中村区名駅１丁目１−４ JRセントラルタワーズ・タワーズプラザ 12階
07. ピッツェリア イゾラ 名古屋ミッドランドスクエア
    評価: 3.4
    住所: 名古屋市中村区名駅４丁目７−１ ミッドランドスクエア 4F
08. 風来坊 近鉄名駅店
    評価: 3.6
    住所: 名古屋市中村区名駅１丁目２−２ 近鉄名古屋駅構内 特急のりば
09. 名古屋 今井屋本店
    評価: 3.8
    住所: 名古屋市中村区名駅４丁目７−１ ミッドランドスクエア 4F
10. 世界の山ちゃん名駅西口店
    評価: 3.8
    住所: 名古屋市中村区名駅１丁目１−１４
11. La maison JOUVAUD KITTE名古屋店
    評価: 3.9
    住所: 名古屋市中村区名駅１丁目１−１ KITTE名古屋1F
12. THE GATEHOUSE
    評価: 4.1
    住所: 名古屋市中村区名駅１丁目１−３ 名古屋JRゲートタワーホテル 15F
13. 名鉄グランドホテル
    評価: 3.8
    住所: 名古屋市中村区名駅１丁目２−４ 名鉄バスターミナルビル １１階～１８階
14. サンマルコ名古屋近鉄店
    評価: 3.9
    住所: 名古屋市中村区名駅１丁目２−２ 近鉄百貨店名古屋店 地下1階
15. スパゲッティ